In [1]:
import cv2
import os
import numpy as np

def load_images_from_directory(directory, target_size=(100, 100)):
    images = []
    filenames = sorted([f for f in os.listdir(directory) if f.endswith(".jpg") or f.endswith(".png")])
    for filename in filenames:
        img_path = os.path.join(directory, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
            img_resized = cv2.resize(img_rgb, target_size)  
            images.append(img_resized)
    return np.array(images)

In [2]:
hr_images=load_images_from_directory('/kaggle/input/my-div2k-dataset/DIV2K_train_HR', (400, 400))
lr_images=load_images_from_directory('/kaggle/input/my-div2k-dataset/DIV2K_train_LR_bicubic/X4')

In [3]:
from keras.models import Model
from keras import Layer
from keras.layers import Dense, Conv2D, PReLU, BatchNormalization, Input, Add, Flatten, Dropout
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

2024-07-29 20:12:17.967658: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 20:12:17.967775: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 20:12:18.144821: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
LR= (100, 100, 3)
HR= (400, 400, 3)

In [5]:
def build_discriminator():
    input_layer = Input(shape=(25, 25, 512))    
    x = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same')(input_layer)
    x = PReLU()(x)    
    x = Conv2D(filters=64, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = PReLU()(x)     
    x = Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = PReLU()(x)    
    x = Conv2D(filters=128, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = PReLU()(x)    
    x = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = PReLU()(x)    
    x = Conv2D(filters=256, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = PReLU()(x)    
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = PReLU()(x)     
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = PReLU()(x)     
    x = Flatten()(x)
    x = Dense(units=1024)(x)
    x = PReLU()(x)    
    x = Dense(units=1, activation='sigmoid')(x)    
    model = Model(inputs=input_layer, outputs=x)
    return model

In [6]:
class MyLayer(Layer):
    def call(self, x):
        return tf.nn.depth_to_space(x, block_size=2)
def build_generator():
    input_layer= Input(shape= LR) #LR= (100, 100, 3), the low resolution image
    x=Conv2D(filters= 64, kernel_size=(9,9), strides=(1,1), padding='same')(input_layer)
    x=PReLU()(x)
    x_shortcut=x
    def add_residual_blocks(input_tensor):
        original_tensor=input_tensor
        input_tensor=Conv2D(filters= 64, kernel_size=(3,3), strides=(1,1), padding='same')(input_tensor)
        input_tensor=BatchNormalization()(input_tensor)
        input_tensor=PReLU()(input_tensor)
        input_tensor=Conv2D(filters= 64, kernel_size=(3,3), strides=(1,1), padding='same')(input_tensor)
        input_tensor=BatchNormalization()(input_tensor)
        input_tensor=Add()([original_tensor, input_tensor])
        return input_tensor
    for i in range(5):
        x=add_residual_blocks(x)
    x=Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(x)
    x=BatchNormalization()(x)
    x=Add()([x, x_shortcut])
    x=Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(x)
    x=MyLayer()(x)#hardcoded function to increase width and height by reducing number of channels
    x=PReLU()(x)
    x=Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(x)
    x=MyLayer()(x)#hardcoded function to increase width and height by reducing number of channels
    x=PReLU()(x)
    x=Conv2D(filters=3, kernel_size=(9,9), strides=(1,1), padding='same')(x)
    model=Model(inputs=input_layer, outputs=x)
    return model

In [7]:
from tensorflow.keras.applications import VGG19
def build_vgg():
    vgg = VGG19(weights="imagenet", include_top=False, input_shape=HR)
    vgg.trainable = False
    model = Model(inputs=vgg.input, outputs=vgg.get_layer("block5_conv4").output)
    return model

In [8]:
def combine(generator, discriminator, vgg):
    discriminator.trainable= False
    input_lr= Input(shape=(100, 100, 3))
    gen_hr=generator(input_lr)
    gen_hr_features=vgg(gen_hr)
    validity=discriminator(gen_hr_features)
    model = Model(inputs=input_lr, outputs=[validity, gen_hr_features])
    model.compile(
        loss=['binary_crossentropy', 'mse'],
        loss_weights=[1e-3, 1],
        optimizer=Adam(0.0002, 0.5)
    )
    return model

In [9]:
import matplotlib.pyplot as plt

In [10]:
from keras import backend as K
import gc
def train(g_model, vgg, d_model, srgan, batch_size=100, epochs=5):
    half_batch_size=int(batch_size/2)
    batches=int(lr_images.shape[0]/batch_size)
    d_model.compile(loss='binary_crossentropy', optimizer='adam')
    c=0
    for epoch in range(epochs):
        print(f'epoch number {epoch+1} has started')
        for batch in range(batches):
            c=c+1
            idx = np.random.randint(0, lr_images.shape[0], half_batch_size)
            
            real_hr_images=hr_images[idx]
            real_hr_features=vgg.predict(real_hr_images)
            real_image_label=np.ones(half_batch_size)
            d_loss_real=d_model.train_on_batch(real_hr_features, real_image_label)
            
            fake_hr_images=g_model.predict(lr_images[idx])
            fake_hr_features=vgg.predict(fake_hr_images)
            fake_image_label=np.zeros(half_batch_size)
            d_loss_fake=d_model.train_on_batch(fake_hr_features, fake_image_label)
            
            indices = np.random.randint(0, lr_images.shape[0], batch_size)    
            images_lr=lr_images[indices]
            images_hr=hr_images[indices]
            images_hr_features=vgg.predict(images_hr)
            gan_label=np.ones(batch_size)
            g_loss=srgan.train_on_batch(images_lr, [gan_label, images_hr_features])
            
            print(f'epoch={epoch+1} batch={batch+1} d_loss on real={d_loss_real} d_loss on fake={d_loss_fake} g_loss={g_loss}')
            
            if (c%100==0):
                img_hr=g_model.predict(lr_images[7])
                plt.imshow(lr_images[7])
                plt.show()
                plt.imshow(hr_images[7])
                plt.show()
            K.clear_session()
            gc.collect()

In [11]:
discriminator=build_discriminator()
generator=build_generator()
vgg=build_vgg()
srgan=combine(generator, discriminator, vgg)
train(generator, vgg, discriminator, srgan)

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
epoch number 1 has started
1/2 ━━━━━━━━━━━━━━━━━━━━ 1:23 84s/step

I0000 00:00:1722284035.900957     109 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2/2 ━━━━━━━━━━━━━━━━━━━━ 137s 53s/step


/opt/conda/lib/python3.10/site-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")
W0000 00:00:1722284097.721772     106 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
2024-07-29 20:15:05.344368: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng35{k2=2,k3=0} for conv (f32[32,3,400,400]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,16,400,400]{3,2,1,0}, f32[3,16,9,9]{3,2,1,0}, f32[3]{0}), window={size=9x9 pad=4_4x4_4}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2024-07-29 20:15:05.489872: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.14559615s
Trying algorithm eng35{k2=2,k3=0} for conv (f32[32,3,4

2/2 ━━━━━━━━━━━━━━━━━━━━ 20s 8s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 858ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 19s 6s/step 


2024-07-29 20:16:18.834374: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng18{k2=1,k4=3,k5=1,k6=0,k7=0} for conv (f32[100,3,400,400]{3,2,1,0}, u8[0]{0}) custom-call(f32[100,16,400,400]{3,2,1,0}, f32[3,16,9,9]{3,2,1,0}, f32[3]{0}), window={size=9x9 pad=4_4x4_4}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2024-07-29 20:16:19.145393: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.311138557s
Trying algorithm eng18{k2=1,k4=3,k5=1,k6=0,k7=0} for conv (f32[100,3,400,400]{3,2,1,0}, u8[0]{0}) custom-call(f32[100,16,400,400]{3,2,1,0}, f32[3,16,9,9]{3,2,1,0}, f32[3]{0}), window={size=9x9 pad=4_4x4_4}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"conv_result_scale":1,"activation_mode":"kNone","side_input_scal

ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code

  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 701, in start

  File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/opt/conda/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 534, in di